In [1]:
import os,sys
import datetime
import numpy as np
import subprocess
import pandas as pd
import xarray as xr

%load_ext autoreload
%autoreload 2

# Estimation of storage

In [2]:
# /work/bb1152/u290372/cesm215_archive/GKLT/initial_piControl/001_0501/atm/hist/001_0501.cam.h1.0501-06-01-00000.nc
size_mb = 323
variables = 17
days = 90
size_per_grid_gb = size_mb / variables / days / 10**3
size_per_grid_gb

0.0002111111111111111

In [3]:
# /work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/piControl-x5/day/atmos/tas/ens001/tas_day_CESM2_piControl-x5_ens001_1850.nc
size_mb = 19
variables = 1
days = 90
size_per_grid_gb = size_mb / variables / days / 10**3
size_per_grid_gb

0.0002111111111111111

In [4]:
# size of daily variable for one year
size_per_grid_gb * 365

0.07705555555555556

In [5]:
# one ensemble (126 members) 90 days 30 variables
size_per_grid_gb * 126 * 90 * 30

71.82

## Re-estimate past computation cost

In [6]:
base = 185404
per_day = 46879

def estimated_node_hours(ndays):
    return (base + ndays * per_day) / 60 / 60 / 256


### preparation of restart files for 1. of June

in March 2025, 126 runs from 1.1 to 15.3 were simulated. Additionally, all summers (126*2) were resimulated

In [7]:
cpuh = 1855
cpuh

1855

In [8]:
estimated_node_hours(73) * 126 + estimated_node_hours(90) * 126 * 2

1697.5818945312499

### summer ensembles simulated in December
in December, 7 full 126 member summer ensembles were simulated

In [9]:
cpuh_1152 = 6798
cpuh_1445 = 1261
cpuh = cpuh_1152 + cpuh_1445
cpuh

8059

In [10]:
estimated_node_hours(5) * 18 * 126 * 7

7231.693710937499

### re-esitmation

In [11]:
from scipy.optimize import minimize

def estimated_error(p):
    def estimated_node_hours_explicit(ndays):
        return (base + ndays * per_day) / 60 / 60 / 256
    base, per_day = p
    
    error = 0
    for i in range(126):
        error += (estimated_node_hours_explicit(73) + estimated_node_hours_explicit(90) * 2 - 1855 / 126) ** 2
    for i in range(126):
        error += (estimated_node_hours_explicit(5) * 18 * 7 - 8059 / 126) ** 2
    return error

In [12]:
res = minimize(estimated_error, [185404, 46879], method='Powell')
res

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 2.607567100433571e-23
       x: [ 2.123e+05  5.111e+04]
     nit: 4
   direc: [[ 0.000e+00  1.000e+00]
           [-1.873e+04  3.257e+03]]
    nfev: 86

In [13]:
base, per_day = res.x
print(f"node hours per started simulations {base:.0f}")
print(f"node hours per simulated day {per_day:.0f}")

node hours per started simulations 212267
node hours per simulated day 51111


# Estimates for requests

## 2025 - October

In [64]:
tab = pd.DataFrame(index=['node hours', 'storage (Gb)'])

In [65]:
n_sims = 18 * 126 * 8
n_days = 5
n_variables = 42
tab['8 EKE summer ensembles - current climate'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [66]:
n_sims = 18 * 126 * 8
n_days = 5
n_variables = 42
tab['8 EKE summer ensembles - pre-industrial'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [67]:
n_sims = 18 * 100 * 5
n_days = 5
n_variables = 42
tab['5 hot summers from wet conditions - current climate'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [68]:
n_sims = 18 * 100 * 5
n_days = 5
n_variables = 42
tab['5 hot summers from wet conditions - pre-industrial'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [69]:
tab['sum'] = tab.sum(axis=1)

In [70]:
tab.round().T

,node hours,storage (Gb)
8 EKE summer ensembles - current climate,9210.0,804.0
8 EKE summer ensembles - pre-industrial,9210.0,804.0
5 hot summers from wet conditions - current climate,4569.0,399.0
5 hot summers from wet conditions - pre-industrial,4569.0,399.0
sum,27558.0,2407.0


In [45]:
18421.0  *2

36842.0

In [ ]:
estimated_node_hours(5) * 72

18420.57142857148

In [49]:
504 * 36

18144

## 2025

In [140]:
tab = pd.DataFrame(index=['node hours', 'storage (Gb)'])

In [ ]:
n_sims = 18 * 126 * 4
n_days = 5
n_variables = 42
tab['4 additional summer ensembles'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [145]:
n_sims = 18 * 126 * 8
n_days = 5
n_variables = 42
tab['8 soilmoisture summer ensembles'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [150]:
n_sims = 24 * 126 * 8
n_days = 7
n_variables = 25
tab['8 low NEP ensembles'] = [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [152]:
# initial condistions
n_sims = 5
n_days = 30
tab['Dunkelflauten'] = [estimated_node_hours(n_days) * n_sims, 15 * n_sims]
# boosting
n_sims = 1000
n_days = 20
n_variables = 25
tab['Dunkelflauten'] += [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [154]:
# initial condistions
n_sims = 126
n_days = 30*5
tab['SH polar vortex'] = [estimated_node_hours(n_days) * n_sims, 15 * n_sims]
# boosting
n_sims = 126 * 3 * 20
n_days = 7
n_variables = 25
tab['SH polar vortex'] += [estimated_node_hours(n_days) * n_sims, size_per_grid_gb * n_sims * n_days * n_variables]

In [163]:
tab.round().T

,node hours,storage (Gb)
4 additional summer ensembles,4605.0,402.0
8 soilmoisture summer ensembles,9210.0,804.0
8 low NEP ensembles,14964.0,894.0
Dunkelflauten,1349.0,181.0
SH polar vortex,5753.0,2169.0


In [160]:
tab.round().T.to_csv('estimate_2025.csv')

## 2024 October estimates

### importance sampling ensemble

In [13]:
N_initial = 42 * 10

In [59]:
d = pd.DataFrame(columns=['description','node hours'])

In [60]:
# prepare restart files
# simulate until end of May
d.loc[len(d.index)] = [f'prepare restart files end of May ({N_initial})', estimated_node_hours(30*5) * N_initial]

In [61]:
# one batch of importance sampling JJA
# 8d periods
d.loc[len(d.index)] = [f'i) importance sampling JJA with tau=8d ({N_initial}) runs' , estimated_node_hours(8) * N_initial * (92 / 8)]
d.loc[len(d.index)] = [f'ii) importance sampling JJA with tau=8d ({N_initial}) runs' , estimated_node_hours(8) * N_initial * (92 / 8)]
d.loc[len(d.index)] = [f'iii) importance sampling JJA with tau=8d ({N_initial}) runs' , estimated_node_hours(8) * N_initial * (92 / 8)]

### stratospheric vortex simulations


In [62]:
# prepare restart files
# simulate until end of September (starting from End of May)
d.loc[len(d.index)] = [f'prepare restart files end of September (20?)', estimated_node_hours(30*4) * 20]

In [63]:
# get early and late vortex start
for i in range(10):
    d.loc[len(d.index)] = [f'{i}) 10 times early & late' , estimated_node_hours(15) * 4 * 50]
    d.loc[len(d.index)] = [f'{i}) finish simulations' , estimated_node_hours(90) * 20]

In [64]:
d

,description,node hours
0,prepare restart files end of May (420),3289.113151
1,i) importance sampling JJA with tau=8d (420) runs,2937.180859
2,ii) importance sampling JJA with tau=8d (420) ...,2937.180859
3,iii) importance sampling JJA with tau=8d (420)...,2937.180859
4,prepare restart files end of September (20?),126.104253
5,0) 10 times early & late,192.836155
6,0) finish simulations,95.584071
7,1) 10 times early & late,192.836155
8,1) finish simulations,95.584071
9,2) 10 times early & late,192.836155


In [67]:
d.iloc[:,1].sum().round()

np.float64(15111.0)

# other

In [ ]:
elapsed_time(365)

4:41:30


In [3]:
estimated_node_hours(360)

18.51328559027778

In [4]:
estimated_node_hours(30) * 12

20.726223958333332

In [5]:
estimated_node_hours(360) / (estimated_node_hours(30) * 12)

0.8932300272107306

In [6]:
sacct_out = subprocess.run('sacct -S 0701 -E0801 -o alloccpus,cputimeraw,elapsed,nnodes,jobname', shell=True, capture_output=True).stdout.strip()
sacct_out = sacct_out.decode('utf8').split('\n')
for line in sacct_out[:10]:
    print(line)

AllocCPUS CPUTimeRAW    Elapsed   NNodes    JobName 
---------- ---------- ---------- -------- ---------- 
      1024   16055296   04:21:19        4 run.b.e21+ 
       256    4013824   04:21:19        1      batch 
      1024   16055296   04:21:19        4     extern 
      1024   16025600   04:20:50        4   cesm.exe 
       256      41472   00:02:42        1 st_archiv+ 
       256      41472   00:02:42        1      batch 
       256      41472   00:02:42        1     extern 
      1024   16258048   04:24:37        4 run.b.e21+ 


In [7]:
seconds = 0
for line in sacct_out:
    if 'cesm.exe' in line:
        seconds += int(line.split()[1])
seconds

1707076608

In [8]:
seconds / 60 / 60 / 256

1852.2966666666666

In [9]:
16055296 / 60 / 60 / 256

17.42111111111111

In [10]:
computation_time(365)

17296239